## The Challenge

You are an asset manager who has recently secured a contract to work with a new small pension fund. The pension fund has decided to switch from their incumbent asset manager on the advice of consultants. The transition is being handled by a transition manager and you have just received the transition accounts with the final confirmed holdings that will make up the initial portfolio for the fund.

As part of the investment agreement with the pension fund they would like to make some changes to the way that they receive their reporting. They would also like to start investing in some exotic over the counter instruments which our internal systems do not support. Therefore in addition to setting up a production portfolio with the initial holdings, you will also need to create a test environment to model these changes without affecting the live environment.

## The Solution

Using LUSID you can mirror your live investment environment into a different segregated scope. This will allow you to continue trading business as usual and driving the best possible performance for your client while simultaneously building and testing our improved reporting and support for more exotic instruments. You can do this by:

1) Creating your live production environment for your new client

2) Creating your instrument universe

3) Adding your take on balances for your client in this live environment

4) Creating your test environment and using derived portfolios to mirror your production environment

5) Making some trades for your client in your production environment and check that they propogate to your test environment

6) Adding some transactions to your test environment and ensuring that they do not affect the live environment

First things first import the libraries that you need and authenticate your LUSID client.

*Run the cell below to import the libraries and authenticate your LUSID client*

In [20]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import json

# Authenticate your user and create your API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.2894.0


![Initialise](img/simplechanges-lusidinit.gif)

## 1) Create your Live Production Environment

In LUSID you can segregate environments by making use of Scopes. Read more about scopes here [LUSID API Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

Each scope has a unique code which is used to identify it. 

*Run the cell below to create a name for your production scope for your new client*

In [21]:
# Generate a unique id for your scope
client_account_id = import_data.create_scope_id()
# Create a name for your scope
production_scope_code = 'production-client-{}'.format(client_account_id)
prettyprint.heading('Production Scope', production_scope_code)

Production Scope: production-client-378d-4585-c5fa-fc


![Scopes](img/simplechanges-prodscope.gif)

To manage your client's assets you will need to set up a portfolio for their fund. Read more about portfolios in LUSID here [LUSID API Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

A portfolio lives within a given scope. Each portfolio also has a unique code. This allows you to uniquely identify a portfolio by the combination of its scope and its code. 

*Run the cell below to create a code for the portfolio which will hold your client's fund*

In [23]:
portfolio_code = 'pension-fund-{}'.format(str(uuid.uuid4())[:4])
prettyprint.heading('Portfolio Code', portfolio_code)

Portfolio Code: pension-fund-34ec


Now that you have a code for your portfolio you can create it in LUSID. For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that the official transer time from the previous asset manager to you was two days ago. You will therefore need to set the creation date of the portfolio to be two days previous. Read more about the importance of the creation date of a portfolio here [LUSID Knowledge Base: Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [24]:
# Set the official transfer time to be 2 days ago
official_transfer_time = (datetime.now(pytz.UTC) - timedelta(days=2))

# Build our request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name='pension-fund',
    description='Pension fund for new client',
    code=portfolio_code,
    created=official_transfer_time,
    base_currency='GBP',
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=None,
    properties=None)

# Call LUSID to create your portfolio
response = client.transaction_portfolios.create_portfolio(
    scope=production_scope_code,
    create_request=request,
)

# Pretty print the response from LUSID
prettyprint.portfolio_response(response)

Portfolio Created
Scope: production-client-378d-4585-c5fa-fc
Code: pension-fund-34ec
Portfolio Effective From: 2019-07-18 17:55:58.366008+00:00
Portfolio Created On: 2019-07-20 17:55:58.579810+00:00



![Pension Fund Portfolio](img/simplechanges-pensionfund.gif)

## 2) Create your Instrument Universe

To take on your new client's assets you need to ensure that you have all the appropriate instruments loaded into your instrument universe. You can read more about instruments in LUSID here [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument). 

In this case you are going to load the instrumets into LUSID via a CSV that has been prepared for you by the transition manager. In reality you would maintain a comprehensive instrument universe and are likely to already have the instruments that this asset manager holds in your instrument master rendering this step unnecessary. 

*Run the cell below to import your instruents from our the CSV file*

In [25]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv('data/instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN
5,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,LN,united_kingdom,GLEN,equity,common_stock,NaN
6,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,LN,united_kingdom,JE/,equity,common_stock,NaN
7,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,LN,united_kingdom,KGF,equity,common_stock,NaN
8,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,LN,united_kingdom,MCRO,equity,common_stock,NaN
9,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,LN,united_kingdom,REL,equity,common_stock,NaN


Now that you have imported the instruments you can upsert them into LUSID. Read more about upsert methods in LUSID here [LUSID Knoweldge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upsert Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [26]:
# Initialise your batch upsert request
batch_upsert_request = {}
# Iterate over your instrument universe
for row, instrument in instrument_universe.iterrows():
    
    # Specify your identifier columns
    identifier_columns = [
        ('isin', 'Isin'), 
        ('figi', 'Figi'), 
        ('ticker', 'Ticker'),
        ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Add the instrument to your batch request
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
                                                                           
# Call LUSID to upsert your batch
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier='Figi')

Instrument Successfully Upserted: UKGiltTreasury_2.0_2025
Figi ID: BBG0088JSC32
LUSID Instrument ID: LUID_62IM971E


Instrument Successfully Upserted: Glencore_LondonStockEx_GLEN
Figi ID: BBG001MM1KV4
LUSID Instrument ID: LUID_7EQLU5N7


Instrument Successfully Upserted: RELXGroup_LondonStockEx_REL
Figi ID: BBG000D03XD4
LUSID Instrument ID: LUID_VXG75IL8


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
Figi ID: BBG000BVPXP1
LUSID Instrument ID: LUID_2DO1PHYP


Instrument Successfully Upserted: EKFDiagnostics_LondonStockEx_EKF
Figi ID: BBG000BVNBN3
LUSID Instrument ID: LUID_23Z5J1XC


Instrument Successfully Upserted: Sage_LondonStockEx_SGE
Figi ID: BBG000BN0PP3
LUSID Instrument ID: LUID_XVL9MYZ4


Instrument Successfully Upserted: WPP_LondonStockEx_WPP
Figi ID: BBG000BF6B57
LUSID Instrument ID: LUID_5MYW4RWM


Instrument Successfully Upserted: MicroFocus_LondonStockEx_MCRO
Figi ID: BBG000G4KKD2
LUSID Instrument ID: LUID_PXPHCD4N


Instrument Successfully Upserted: Whitebread_Lond

![Instrument Master](img/simplechanges-instrumentmaster.gif)

## 3) Add your Take on Balances

Now that you have populated your instrument universe you can take on the holdings that will make up the new client's fund.

The details of these holdings have been sent to you by the transition manager in a CSV file. 

*Run the cell below to import your take on balances from the CSV file*

In [27]:
holdings = pd.read_csv('data/simplechanges-holdings.csv')
holdings.head(n=10)

,instrument_name,quantity,price,figi,client_internal,currency
0,WPP_LondonStockEx_WPP,1050200,8.7100,BBG000BF6B57,imd_34536734,GBP
1,UKGiltTreasury_3.5_2045,100674,129.9760,BBG006N6HZM7,imd_54234532,GBP
2,UKGiltTreasury_2.0_2025,220456,106.4210,BBG0088JSC32,imd_34534536,GBP
3,UKGiltTreasury_4.5_2034,97658,138.4660,BBG0000D14P3,imd_34534534,GBP
4,UKGiltTreasury_3.75_2021,320568,108.1750,BBG001KKJLR4,imd_34643653,GBP
5,Kingfisher_LondonStockEx_KGF,2879200,2.3940,BBG000BKH1W6,imd_34535552,GBP
6,JustEat_LondonStockEx_JE,4789100,5.9780,BBG0065YWM39,imd_35436366,GBP
7,RELXGroup_LondonStockEx_REL,6134800,16.5750,BBG000D03XD4,imd_43532542,GBP
8,TESCO_LondonStockEx_TSCO,13456500,1.9365,BBG000BF46Y8,imd_34634673,GBP
9,Whitebread_LondonStockEx_WTB,8954300,4.5690,BBG000BRVH05,imd_35430090,GBP


The official transfer took place 2 days ago. You therefore want to load your holdings into your portfolio to be effective from this date. 

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings)

*Run the cell below to set the holdings on your client's portfolio*

In [28]:
# Initialise your holding adjustments list
holding_adjustments = []

# Create a holding adjustment for each instrument
for index, holding in holdings.iterrows():

    # Set the identifier based on whether the instrument is cash or not
    if 'Cash' in holding['instrument_name']:
        identifier_key = 'Instrument/default/Currency'
        identifer = instrument_name.split('_')[0]
    else:
        identifier_key = 'Instrument/default/Figi'
        identifier = holding['figi']

    # Build your holding adjustment request and append it to your list
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                identifier_key: identifier},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=holding['quantity'],
                    cost=models.CurrencyAndAmount(
                        amount=holding['quantity'] * holding['price'],
                        currency=holding['currency']),
                    portfolio_cost=holding['quantity'] * holding['price'],
                    price=holding['price'])
            ]
        )
    )
    
# Call LUSID to set the holdings on your portfolio
response = client.transaction_portfolios.set_holdings(
    scope=production_scope_code,
    code=portfolio_code,
    effective_at=official_transfer_time,
    holding_adjustments=holding_adjustments)

# Pretty print the response from LUSID
prettyprint.set_holdings_response(response, production_scope_code, portfolio_code)

Holdings Successfully Set for Portfolio
Scope: production-client-378d-4585-c5fa-fc
Code: pension-fund-34ec
Holdings Effective From: 2019-07-18 17:55:58.366008+00:00
Holdings Created On: 2019-07-20 17:56:19.876525+00:00



![Pension Fund Portfolio](img/simplechanges-productionholdings.gif)

## 4) Create your Test Environment

Now that you have created our portfolio in your live trading environment you need to create a test environment to work on building the improved reporting and instrument support that the client has requested. 

To do this you will create a new scope for testing purposes.

*Run the cell below to create a unique code for your testing scope*

In [44]:
test_scope_code = 'test-client-{}'.format(client_account_id)
prettyprint.heading('Test Scope', production_scope_code)

Test Scope: production-client-378d-4585-c5fa-fc


![Test Scope](img/simplechanges-testscope.gif)

You can now create a one-way-sync between your portfolio in your production environment and a new derived portfolio in your test environment. Read more about derived portfolios here [LUSID API Knowledge Base: Derived Portfolios](https://support.lusid.com/what-is-a-derived-portfolio-and-how-are-they-used).

For further usage of the create derived portfolio API call refer to the [LUSID API Docs: Create a Derived Portfolio](https://docs.lusid.com/#operation/CreateDerivedPortfolio).

*Run the cell below to create your derived portfolio in the test environment*

In [30]:
# Build your request to create our derived portfolio
request = models.CreateDerivedTransactionPortfolioRequest(
    display_name='pension-fund',
    description='Pension fund for new client',
    code=portfolio_code,
    parent_portfolio_id=models.ResourceId(
        scope=production_scope_code,
        code=portfolio_code),
    created=official_transfer_time)
  
# Call LUSID to create your derived portfolio
response = client.derived_transaction_portfolios.create_derived_portfolio(
    scope=test_scope_code,
    portfolio=request)
    
# Pretty print the response from LUSID
prettyprint.derived_portfolio_response(response)

Portfolio Created
Scope: test-client-378d-4585-c5fa-fc
Code: pension-fund-34ec
Parent Portfolio Scope: production-client-378d-4585-c5fa-fc
Parent Portfolio Code: pension-fund-34ec
Portfolio Effective From: 2019-07-18 17:55:58.366008+00:00
Portfolio Created On: 2019-07-20 17:56:32.381740+00:00



![Derived Portfolio](img/simplechanges-derivedportfolio.gif)

You can check that the derived portfolio has inherited all of the holdings of the parent by retrieving the holdings from each of your portfolios and confirming that they are the same.

For further usage of the get holdings API call refer to the [LUSID API Docs: Get Holdings](https://docs.lusid.com/#operation/GetHoldings).

You can start by getting the holdings from our portfolio in the production scope.

*Run the cell below to get the holdings from your portfolio in the production scope*

In [45]:
# Call LUSID to get your holdings from the portfolio in producton
response = client.transaction_portfolios.get_holdings(
    scope=production_scope_code,
    code=portfolio_code,
    effective_at=official_transfer_time,
    property_keys=['Instrument/default/Name'])

# Pretty print the response from LUSID
prettyprint.holdings_response(response, production_scope_code, portfolio_code)

Holdings for Portfolio
Scope: production-client-378d-4585-c5fa-fc
Code: pension-fund-34ec

Instrument/default/Name: Whitebread_LondonStockEx_WTB
Units: 8954300.0
Cost: 40912196.7
Currency: GBP




![Derived Portfolio](img/simplechanges-prodholdings.gif)

Now let's retrieve your holdings from the derived portfolio in the test scope.

*Run the cell below to retrieve your holdings from the derived portfolio in the test scope*

In [36]:
# Call LUSID to get your holdings from the portfolio in the test environment
response = client.transaction_portfolios.get_holdings(
    scope=test_scope_code,
    code=portfolio_code,
    effective_at=official_transfer_time,
    property_keys=['Instrument/default/Name'])

# Pretty print the response from LUSID
prettyprint.holdings_response(response, test_scope_code, portfolio_code)

Holdings for Portfolio
Scope: test-client-378d-4585-c5fa-fc
Code: pension-fund-34ec

Instrument/default/Name: Whitebread_LondonStockEx_WTB
Units: 8954300.0
Cost: 40912196.7
Currency: GBP




![Derived Portfolio](img/simplechanges-testholdings.gif)

## 5) Make some Live Trades in your Production Environment

Even though you have just added the client's portfolio into LUSID you have already made some trades for your new client to ensure that you can outperform their previous manager. You can import these trades from a CSV file. Note that in practice these would ideally come from an order or execution management system. 

*Run the cell below to import our transactions*

In [37]:
transactions = pd.read_csv('data/simplechanges-transactions.csv')
transactions.head()

,transaction_id,instrument_uid,type,units,transaction_price,transaction_currency,figi,client_internal
0,tid_35b77104-7c72-4515-af2b-d22bb45e6edd,WPP_LondonStockEx_WPP,Sell,410000,8.3720,GBP,BBG000BF6B57,imd_34536734
1,tid_4f192c6a-e74b-464e-95c2-62a18b092bd8,Kingfisher_LondonStockEx_KGF,Sell,2879200,2.3630,GBP,BBG000BKH1W6,imd_34535552
2,tid_3a95473f-1b4a-4652-8047-91624f3f0e19,Glencore_LondonStockEx_GLEN,Buy,669180,2.7385,GBP,BBG001MM1KV4,imd_34534555


Now that you have imported your trades from yesterday you can add them to LUSID. Note that for simplicity all trades have the same transaction date and settlement date which you can see in the request below. Read more about transactions in LUSID here [LUSID Knowledge Base: Transactions](https://support.lusid.com/what-is-a-transaction). 

For further usage of the upsert transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://docs.lusid.com/#operation/UpsertTransactions).

*Run the cell below to upsert your trades into LUSID*

In [39]:
# Initialise a list to hold your requests
requests = []

yesterday = datetime.now(pytz.UTC)-timedelta(days=1)
tomorrow = datetime.now(pytz.UTC)+timedelta(days=1)

# Iterate over your transactions
for index, transaction in transactions.iterrows():
    
    # Build your request and append it to your list
    requests.append(
        models.TransactionRequest(
            transaction_id=transaction['transaction_id'],
            type=transaction['type'],
            instrument_identifiers={
                'Instrument/default/Figi': transaction['figi']},
            transaction_date=yesterday,
            settlement_date=tomorrow,
            units=transaction['units'],
            transaction_price=models.TransactionPrice(
                price=transaction['transaction_price'],
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=transaction['transaction_price'] * transaction['units'],
                currency=transaction['transaction_currency']),
            source='Client',
            transaction_currency=transaction['transaction_currency']))

# Call LUSID to upsert your transactions
response = client.transaction_portfolios.upsert_transactions(
    scope=production_scope_code,
    code=portfolio_code,
    transactions=requests)

# Pretty print the response from LUSID
prettyprint.transactions_response(
    response,
    production_scope_code,
    portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: production-client-378d-4585-c5fa-fc
Code: pension-fund-34ec
Transactions Effective From: 2019-07-19 17:58:54.958703+00:00
Transactions Created On: 2019-07-20 17:58:55.510586+00:00



As you have created a derived portfolio these trades will also appear in your test environment. You can check this by retrieving the transactions from your portfolio in the test environment. 

For further usage of the get transactions API call refer to the [LUSID API Docs: Get Transactions](https://docs.lusid.com/#operation/GetTransactions).

*Run the cell below to get your transactions from the test environment*

In [40]:
# Call LUSID to get all transactions made yesterday
response = client.transaction_portfolios.get_transactions(
    scope=test_scope_code,
    code=portfolio_code,
    from_transaction_date=yesterday,
    to_transaction_date=yesterday,
    property_keys=['Instrument/default/Name']
)

# Pretty print the response from LUSID
prettyprint.get_transactions_response(
    response, 
    test_scope_code, 
    portfolio_code, 
    property_keys=['Instrument/default/Name'])

Transactions Retrieved from Portfolio
Scope:  test-client-378d-4585-c5fa-fc
Code:  pension-fund-34ec 

Transaction Id:  tid_35b77104-7c72-4515-af2b-d22bb45e6edd
Transaction Type:  Sell
Units:  410000.0
Price:  8.372
Currency:  GBP
Instrument/default/Name: WPP_LondonStockEx_WPP
Transaction Date:  2019-07-19 17:58:54.958703+00:00 

Transaction Id:  tid_4f192c6a-e74b-464e-95c2-62a18b092bd8
Transaction Type:  Sell
Units:  2879200.0
Price:  2.363
Currency:  GBP
Instrument/default/Name: Kingfisher_LondonStockEx_KGF
Transaction Date:  2019-07-19 17:58:54.958703+00:00 

Transaction Id:  tid_3a95473f-1b4a-4652-8047-91624f3f0e19
Transaction Type:  Buy
Units:  669180.0
Price:  2.7385
Currency:  GBP
Instrument/default/Name: Glencore_LondonStockEx_GLEN
Transaction Date:  2019-07-19 17:58:54.958703+00:00 



![Transactions Production Environment](img/simplechanges-transactionsprod.gif)

## 6) Add some Test Transactions in your Test Environment

You now want to add a transaction to your test environment so that your reporting team can test some new functionality on over the counter (OTC) contracts. You will import a overly simplified sample contract from a JSON file. 

*Run the cell below to import your simple OTC swap contract*

In [41]:
# Import our simple over the counter swap contract from a JSON file
simple_otc_swap_contract = open('data/interest_rate_swap.json').read()
simple_otc_swap_contract = json.loads(simple_otc_swap_contract)
# Pretty print our contract
pprint.pprint(simple_otc_swap_contract)

{'InternalContractId': 'irs_85678992',
 'MaturityDate': '04/04/2025',
 'StartDate': '03/04/2015',
 'assetType': 'swap',
 'flow conventions': 'GBP_6m_LIBOR',
 'strike': 0.02234}


Now that you have loaded your contract you need to add it to the instrument master in LUSID. Read more about adding OTC instruments to LUSID here [LUSID Knowledge Base: Creating Bespoke Instruments](https://support.lusid.com/creating-bespoke-instruments).

*Run the cell below to upsert your OTC contract into LUSID*

In [42]:
# Create the defintion for your instrument
simple_otc_swap_instrument = models.InstrumentDefinition(
    name=simple_otc_swap_contract['InternalContractId'],
    identifiers={
        'ClientInternal': models.InstrumentIdValue(
            value=simple_otc_swap_contract['InternalContractId'])
    },
    definition=models.InstrumentEconomicDefinition(
        instrument_format='json',
        content=json.dumps(simple_otc_swap_contract)),
    properties=None)

# Call LUSID to upsert your OTC swap contract
instrument_response = client.instruments.upsert_instruments(
    requests={'simple_otc_swap_instrument': simple_otc_swap_instrument})

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response)

Instrument Successfully Upserted: simple_otc_swap_instrument
ClientInternal ID: irs_85678992
LUSID Instrument ID: LUID_D4AY0P56


1  instruments upserted successfully
0  instrument upsert failures


Now that you have your OTC contract added to the LUSID instrument master you can book a trade against it on your derived portfolio in your test environment.

*Run the cell below to upsert your OTC transaction into the derived portfolio*

In [43]:
in_two_days = datetime.now(pytz.UTC)+timedelta(days=2)
in_four_days = datetime.now(pytz.UTC)+timedelta(days=4)

request = models.TransactionRequest(
    transaction_id='tid_35b77104-7c72-4515-af2b-d22bb45e6eff',
    type='Buy',
    instrument_identifiers={
        'Instrument/default/ClientInternal': simple_otc_swap_contract['InternalContractId']},
    transaction_date=in_two_days,
    settlement_date=in_four_days,
    units=1,
    transaction_price=models.TransactionPrice(
        price=0,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=0,
        currency='GBP'),
    source='Client',
    transaction_currency='GBP')

# Call LUSID to upsert your transactions
response = client.transaction_portfolios.upsert_transactions(
    scope=test_scope_code,
    code=portfolio_code,
    transactions=[request])

# Pretty print the response from LUSID
prettyprint.transactions_response(response, test_scope_code, portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: test-client-378d-4585-c5fa-fc
Code: pension-fund-34ec
Transactions Effective From: 2019-07-22 17:59:10.562207+00:00
Transactions Created On: 2019-07-20 17:59:10.913564+00:00



Now you can ensure that adding this trade has not affected the production environment. 

*Run the cell below to check to see if any of the transactions can be found in the production environment*

In [19]:
# Call LUSID to see if the transaction from the test environment propogated through
response = client.transaction_portfolios.get_transactions(
    scope=production_scope_code,
    code=portfolio_code,
    from_transaction_date=in_two_days,
    to_transaction_date=in_two_days,
    property_keys=['Instrument/default/Name'])

# Pretty print the response from LUSID
prettyprint.get_transactions_response(
    response, 
    production_scope_code, 
    portfolio_code,
    ['Instrument/default/Name'])

Transactions Retrieved from Portfolio
Scope:  production-client-3768-40b3-11fd-b4
Code:  pension-fund-2d92 



![Transactions Production Environment](img/simplechanges-transactionstest.gif)

Now you have both a live environment and a test environment that are linked via a one-way sync. This ensures that any changes to the live environment and prevents you from having to worry about keeping the two in sync. In addition if you need to make any trades to your testing environment you can do this with the assurance that they will not affect your live trading environement. 